# Web Page Summary

#### This module summarises the web page using the Ollama 3.2:1b LLM

    

In [1]:
# import modules 

import os 
import requests
from dotenv import load_dotenv
from IPython.display import display,Markdown
from bs4 import BeautifulSoup
from openai import OpenAI


In [3]:
# openai = OpenAI(base_url = 'http://localhost:11434/v1', api_key = 'ollama')
load_dotenv(override=True)
api_key = os.getenv('PERPLEXITY_API_KEY')
openai = OpenAI(base_url="https://api.perplexity.ai" , api_key=api_key)

In [4]:
# chceking the llm
system_prompt = ' you are a helpful assistant'
user_prompt = 'what is 2+2?'
messages= [
    {"role":"system","content": system_prompt},
    {"role":"user","content":user_prompt}
]

In [8]:
response = openai.chat.completions.create(model = 'sonar', messages = messages)
print(response.choices[0].message.content)

**2 + 2 equals 4.**

This is the standard result of basic arithmetic addition in standard mathematics, where adding the integer 2 to itself yields 4.[2][5] Sources discussing order of operations or more complex expressions (like division or multiplication) do not alter this direct sum, as no such operations apply here.[1][3] Cultural or literary references to "2 + 2 = 5" represent fallacies, satire, or rounding approximations, not literal math.[4]


### Summary Project


In [9]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    def __init__(self, url) -> None:
        self.url = url
        response = requests.get(url,headers=headers)
        soup = BeautifulSoup(response.content,'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelavent in soup.body(["script", "style", "img", "input"]):
            irrelavent.decompose()
        self.text = soup.body.get_text(separator= "\n", strip=True)
        

In [10]:
web_content = Website("https://www.google.com/")
print(web_content.title)
print(web_content.text)

Google
About
Store
Gmail
Images
Sign in
Upload image
Upload file
AI Mode
See more
Delete
Delete
Report inappropriate predictions
Cannot upload. Use a file in one of these formats: .avif, .bmp, .jpeg, .pdf, .png, .webp”
Google offered in:
हिन्दी
বাংলা
తెలుగు
मराठी
தமிழ்
ગુજરાતી
ಕನ್ನಡ
മലയാളം
ਪੰਜਾਬੀ
India
Advertising
Business
How Search works
Privacy
Terms
Settings
Search settings
Advanced search
Your data in Search
Search history
Search help
Send feedback
Dark theme: Off
Google apps


Prompts

In [11]:
system_prompt = """
    you are an assistant that analyses the content of the web page\
    and provide an short summary, ignore the text that is related  to navigation \
    response in markdown  
"""

In [12]:
# user_prompt 
def user_prompt(web_content):
    user_prompt = f'you are looking at the website titled {web_content.title}'
    user_prompt+= """\n The contents of the website is as follows\
    provide a short summary about this website in markdown format\
    if it include news or anownsments summarise those as well
    """
    user_prompt+=web_content.text
    return user_prompt

In [13]:
def messages_for(website):
    messages = [
        {"role":"system","content":system_prompt},
        {"role":"user","content":user_prompt(website)}
    ]
    return messages

In [28]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarise_url(website):
    web_content = Website(website)
    response = openai.chat.completions.create(
        model="sonar", 
        messages= messages_for(web_content)
        )
    return response.choices[0].message.content


In [29]:
summarise_url("https://edwarddonner.com")

'# Edward Donner\'s Personal Website Summary\n\n**Edward Donner\'s website** is a personal platform showcasing his work in AI engineering, education, and recruitment technology. The site serves as a hub for his professional projects, courses, and insights.\n\n## Key Content Areas\n\n- **AI Curriculum & Courses**: Donner offers comprehensive AI courses through Udemy with 400,000 enrolled students across 190 countries, focusing on practical AI engineering skills[2]\n- **Nebula.io**: His current venture as **co-founder and CTO**, applying AI to talent recruitment and helping people discover career potential[2]\n- **Interactive Projects**: Includes "Connect Four" and "Outsmart," an arena where large language models compete in strategic games\n- **Professional Background**: Highlights his transition from Wall Street (15-year tenure leading technology teams, including as Managing Director at JPMorgan Chase) to AI entrepreneurship[1]\n\n## Recent Activity & Announcements\n\nThe site features 

In [30]:
# To display content properlu

def display_summary(url):
    summary = summarise_url(url)
    display(Markdown(summary))

In [31]:
display_summary("https://git.com")

## CompanyAI Website Summary

**CompanyAI** is a forthcoming AI platform described as a "superintelligent layer" and "Personal OS" for modern life and work, featuring interconnected apps like **How.com** (trustworthy question-answering), **Email.com** (AI email assistant for inbox zero), **Face.com**, **Git.com**, **Notebook.com** (knowledge retrieval), and **Queen.com** (energy/time management).

- **Status**: "COMING SOON" with "Initializing..." and "Sync with us" prompts; marked as **NEW** for key apps.
- **Key Features**: Shared AI "brain" that learns user patterns across apps, productivity tools turning routines into "power moves," trusted by teams, testimonials from users (e.g., UX designers, PMs praising integration and efficiency).
- **Philosophy**: Built for humans with AI—serious for work, human for life; long-term design, simple controls, shared memory; contact via ping@company.ai.

No specific news or announcements beyond the "COMING SOON" launch teaser.